In [15]:
import gym
import random
from random import randrange

env = gym.make("Taxi-v3", render_mode = 'ansi').env
state, info = env.reset(seed = 9)

# Initially, the taxi needs to pick-up the passenger first
shouldPickUp = True
shouldDropOff = False

numOfSuccesses=0


done = False


def renderMap():
    ok = env.render()
    print(ok)  

#get 6 states
def get_pickup_state():
    state = env.s
    taxi_row, taxi_col, pass_loc, dest_idx = env.decode(state)
    pickup_loc = env.locs[pass_loc]
    dropoff_loc = env.locs[dest_idx]
    
    # Calculate distances from taxi to pick-up location and from pick-up to drop-off location
    # dist_pickup = abs(taxi_row - pickup_loc[0]) + abs(taxi_col - pickup_loc[1])
    # dist_dropoff = abs(pickup_loc[0] - dropoff_loc[0]) + abs(pickup_loc[1] - dropoff_loc[1])
    
    # Return taxi location, drop-off location, pick-up location, and distances
    return taxi_row, taxi_col, dropoff_loc, pickup_loc

#get 5 states
def get_dropoff_state():

    state = env.s
    taxi_row, taxi_col, pass_loc, dest_idx = env.decode(state)
    dropoff_loc = env.locs[dest_idx]
    
    # Calculate distances from taxi to pick-up location and from pick-up to drop-off location
    # dist_pickup = abs(taxi_row - pickup_loc[0]) + abs(taxi_col - pickup_loc[1])
    # dist_dropoff = abs(pickup_loc[0] - dropoff_loc[0]) + abs(pickup_loc[1] - dropoff_loc[1])
    
    # Return taxi location, drop-off location, pick-up location, and distances
    return taxi_row, taxi_col, dropoff_loc

def print_state():
    taxi_row, taxi_col, dropoff_loc, pickup_loc= get_pickup_state()

    
    print("Taxi row:", taxi_row)
    print("Taxi column:", taxi_col)
    print("Drop-off location:", dropoff_loc)
    print("Pick-up location:", pickup_loc)


# There are 6 discrete deterministic actions:
# - 0: move south
# - 1: move north
# - 2: move east
# - 3: move west
# - 4: pickup passenger
# - 5: drop off passenger


# - 0: move south
def P0():
    if SHOW_DETAILS:
        print("P0 - move south")
    env.step(0)
  
#     print("================")
    return "1"

# - 1: move north
def P1():
    if SHOW_DETAILS:
        print("P1 - move north")
    env.step(1)
#     print("================")
    return "1"

# - 2: move east
def P2():
    if SHOW_DETAILS:
        print("P2 - move east")
    env.step(2)
#     print("================")
    return "1"

# - 3: move west
def P3():
    if SHOW_DETAILS:
        print("P3 - move west")
    env.step(3)
#     print("================")
    return "1"

# - 4: pickup passenger
def P4():
#     print("Pick-up passenger")
    global shouldPickUp
    global shouldDropOff
    env.step(4)
    shouldPickUp = False
    shouldDropOff = True
    
    if SHOW_DETAILS:
        print("Passenger has been picked-up")
    return "1"

# - 5: drop off passenger
def P5():
    global done
    global numOfSuccesses
#     print("drop-off passenger")
    new_state = env.step(5)
    if SHOW_DETAILS:
        print("Dropped-off passenger successfully")
        print("======================================")
    done = True
    numOfSuccesses += 1
    return "1"

#Check if had picked up 
def J1():
    if SHOW_DETAILS:
        print("J1")
   
    if shouldPickUp == True and shouldDropOff == False:
        return "3"
    elif shouldPickUp == False and shouldDropOff == True:
        return "2"

#drop off branch
def J2():
    if SHOW_DETAILS:
        print("J2")
    if shouldPickUp == True and shouldDropOff == False:
        taxi_row, taxi_col, dropoff_loc, pickup_loc = get_pickup_state()
    elif shouldPickUp == False and shouldDropOff == True:
        taxi_row, taxi_col, dropoff_loc= get_dropoff_state()
    if dropoff_loc[0] == taxi_row and dropoff_loc[1] == taxi_col:
        return "11"
    elif dropoff_loc[1] > taxi_col:
        return "4"
    elif dropoff_loc[1] < taxi_col:
        return "5"
    elif dropoff_loc[0] < taxi_row:
       return "7"
    elif dropoff_loc[0] > taxi_row:
        return"6"

#pick up branch
def J3():
    if SHOW_DETAILS:
        print("J3")
    taxi_row, taxi_col, dropoff_loc, pickup_loc = get_pickup_state()
    # ok = []
    # # print("=============")
    # # print(pickup_loc[0])
    # # print(taxi_row)
    # if pickup_loc[0] < taxi_row:  # Move North
    #     ok.append("P1")
    # elif pickup_loc[0] > taxi_row:  # Move South
    #     ok.append("P0")
    # elif pickup_loc[1] > taxi_col:  # Move East
    #     ok.append("J4")
    # elif pickup_loc[1] < taxi_col:  # Move West
    #     ok.append("J5")
    
    # if ok:
    #     print("$$$")
    #     print(random.choice(ok))
    #     return random.choice(ok)
    # else:
    #     return "P4"
    if pickup_loc[0] == taxi_row and pickup_loc[1] == taxi_col:  # Pickup location is same as taxi location
        return "10"
    elif pickup_loc[1] > taxi_col:  # Move East
        return "4" 
    elif pickup_loc[1] < taxi_col:  # Move West
        return "5"
    elif pickup_loc[0] > taxi_row:  # Move South
        return "6"
    elif pickup_loc[0] < taxi_row:  # Move North
        return "7"

#move east branch
def J4():
    if SHOW_DETAILS:
        print("J4")
    if shouldPickUp == True and shouldDropOff == False:
        taxi_row, taxi_col, dropoff_loc, pickup_loc = get_pickup_state()
    elif shouldPickUp == False and shouldDropOff == True:
        taxi_row, taxi_col, dropoff_loc= get_dropoff_state()

    specific_locations = [(0, 1), (1, 1), (3, 0), (4, 0), (3, 2), (4, 2)]

    if (taxi_row, taxi_col) in specific_locations:
        # Check if increasing taxi_row and the new location is still in the specific locations
        if taxi_row < 2:
            return "6"#move south
        else:
            return "7"#move North
    return "8"

#move west branch
def J5():
    
    if SHOW_DETAILS:
        print("J5")
    if shouldPickUp == True and shouldDropOff == False:
        taxi_row, taxi_col, dropoff_loc, pickup_loc = get_pickup_state()
    elif shouldPickUp == False and shouldDropOff == True:
        taxi_row, taxi_col, dropoff_loc= get_dropoff_state()
    specific_locations = [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]
    if (taxi_row, taxi_col) in specific_locations:
        # Check if increasing taxi_col and the new location is still in the specific locations
        if taxi_row < 2:
            return "6"  #move south
        else:
            return "7"  #move North
    return "9"

class Node:
    def __init__(self, ID, node_type, subtype, delay, *connections):
        self.ID = ID
        self.type = node_type
        self.subtype = subtype
        self.delay = delay
        self.connections = connections
        self.next = None

nodeList = []
for i in range(1, 6):
    node_id = "J" + str(i)
    node_type = 1
    subtype = i
    delay = 1
    if i == 1:
        connections = ["J2", "J3"]
    elif i == 2:
        connections = ["P1", "P0", "J4", "J5", "P5"]
    elif i == 3:
        connections = ["P1", "P0", "J4", "J5", "P4"]
    elif i == 4:
        connections = ["P0", "P1", "P2"]
    elif i == 5:
        connections = ["P0", "P1", "P3"]
    node = Node(node_id, node_type, subtype, delay, *connections)
    nodeList.append(node)

# for node in nodeList:
#      print(f"ID: {node.ID}, Type: {node.type}, Subtype: {node.subtype}, TimeDelay: {node.delay}, Connections: {node.connections}")

# functionsLibrary = {
#     "J1": J1,
#     "J2": J2,
#     "J3": J3,
#     "J4": J4,
#     "J5": J5,
#     "P0": P0,
#     "P1": P1,
#     "P2": P2,
#     "P3": P3,
#     "P4": P4,
#     "P5": P5
# }

functionsLibrary = {
    "1": J1,
    "2": J2,
    "3": J3,
    "4": J4,
    "5": J5,
    "6": P0,
    "7": P1,
    "8": P2,
    "9": P3,
    "10": P4,
    "11": P5
}


totalTimeDelay = 0
trials = 1
solutions = []  # Initialize an empty list to store solutions
random.seed(a=777, version=2)
MAX_TIME = 66  # 42 #40, originally, but fails one trial
numOfSuccesses = 0
SHOW_DETAILS = False  # True
GENERATE_RANDOM_ENV = False

if GENERATE_RANDOM_ENV:
    print("Experiment with RANDOM ENVIRONMENTS")
else:
    print("Experiment with Deterministic environments")

while trials <= 7:
    
    if GENERATE_RANDOM_ENV:
        r = randrange(7)
        info = env.reset(seed=r)
    else:
        info = env.reset(seed=trials)
    start = "1"
    solution = []  # This will store the solution for the current trial
    total = 0
    current = "1"
    
    # Initially, the taxi needs to pick-up the passenger first - newly added
    shouldPickUp = True
    shouldDropOff = False
    done = False
    
    while total < MAX_TIME and not(done):
  
        current_function = functionsLibrary[current]
        returned_value = current_function()  # add fitness computations here, based on interaction with environment
        total += 1
        solution.append(current)  # Add the current step to the solution
        current = returned_value
        
    solutions.append(solution)  # Add the solution for this trial to the solutions list

    trials += 1


# Writing solutions to a text file, each solution on a single line
with open('solutions.txt', 'w') as file:
    for solution in solutions:
        # Join the elements of each solution with a comma and write to the file
        solution_line = ','.join(map(str, solution))  # Converts each element to string and joins with commas
        file.write(solution_line + "\n")  # Writes the joined string to the file, then a newline character

    

Experiment with Deterministic environments
